In [8]:
import numpy as np
import matplotlib.pyplot as pyplot
import pandas as pd
import os
from datetime import datetime, timedelta
import random

In [9]:
# step 1: get or simulate some data


In [10]:

OUTPUT_DIR = "synthetic_synergy_data"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ------------------------------------------------------------
# PARAMETERS
# ------------------------------------------------------------
N_CUSTOMERS = 50_000      # change to 1_170_000 when needed
N_SMART_METERS = 12_000   # subset for interval data
N_YEARS = 3

np.random.seed(42)


# ============================================================
# A. BILLING SYSTEM DATA
# ============================================================

def simulate_billing_system_data():
    tariffs = ["A1", "L1", "L3", "R1", "R3", "M1", "K1"]
    customer_types = ["residential", "small_business", "medium_business", "large_business", "home_business"]

    df = pd.DataFrame({
        "customer_id": np.arange(1, N_CUSTOMERS + 1),
        "postcode": np.random.randint(6000, 6999, N_CUSTOMERS),
        "suburb_code": np.random.randint(1, 500, N_CUSTOMERS),
        "tariff": np.random.choice(tariffs, N_CUSTOMERS),
        "customer_type": np.random.choice(customer_types, N_CUSTOMERS),
        "monthly_consumption_kwh": np.random.gamma(shape=2.0, scale=200, size=N_CUSTOMERS),  # realistic heavy-tail
        "bill_amount": np.random.uniform(80, 600, N_CUSTOMERS),
        "payment_history_score": np.random.uniform(0, 1, N_CUSTOMERS),
        "concession_status": np.random.choice([0, 1], N_CUSTOMERS, p=[0.85, 0.15]),
        "connection_date": pd.date_range("2010-01-01", periods=N_CUSTOMERS).to_series().sample(N_CUSTOMERS).values,
    })

    df["customer_tenure_years"] = (pd.Timestamp("2025-01-01") - pd.to_datetime(df["connection_date"])).dt.days / 365

    df.to_csv(f"{OUTPUT_DIR}/billing_system_data.csv", index=False)
    print("Saved billing_system_data.csv")
    return df



# ============================================================
# B. ADVANCED METER INTERVAL DATA (15/30 MIN)
# ============================================================

def simulate_interval_data(customer_ids):
    interval_records = []

    for cid in customer_ids:
        for day in range(0, 7):  # one week simulation, scale if needed
            dt_start = datetime(2024, 1, 1) + timedelta(days=day)
            for i in range(48):  # 30-min intervals
                timestamp = dt_start + timedelta(minutes=30 * i)
                kwh = max(0, np.random.normal(0.5, 0.3))  # realistic low usage per interval
                solar = max(0, np.random.normal(0.3, 0.2)) if 10 <= timestamp.hour <= 15 else 0

                interval_records.append([cid, timestamp, kwh, solar])

    df = pd.DataFrame(interval_records,
                      columns=["customer_id", "timestamp", "kwh", "solar_export_kwh"])

    df.to_csv(f"{OUTPUT_DIR}/advanced_interval_data.csv", index=False)
    print("Saved advanced_interval_data.csv")
    return df



# ============================================================
# C. REGULATORY REPORTING DATA (ANNUAL)
# ============================================================

def simulate_regulatory_reporting():
    data = []
    for year in [2022, 2023, 2024]:
        data.append({
            "year": year,
            "total_customers": N_CUSTOMERS,
            "residential_customers": int(N_CUSTOMERS * 0.8),
            "business_customers": int(N_CUSTOMERS * 0.2),
            "complaints": np.random.randint(5000, 15000),
            "affordability_index": np.random.uniform(0.2, 0.8),
            "disconnections": np.random.randint(2000, 5000),
            "payment_difficulty_cases": np.random.randint(5000, 15000),
            "life_support_customers": np.random.randint(500, 1500),
        })

    df = pd.DataFrame(data)
    df.to_csv(f"{OUTPUT_DIR}/regulatory_reporting.csv", index=False)
    print("Saved regulatory_reporting.csv")
    return df



# ============================================================
# D. TECHNOLOGY ADOPTION DATA (Solar, EV, Batteries)
# ============================================================

def simulate_technology_adoption(customer_ids):
    df = pd.DataFrame({
        "customer_id": customer_ids,
        "solar_capacity_kw": np.random.choice([0, 3, 6, 10], size=len(customer_ids), p=[0.6, 0.2, 0.15, 0.05]),
        "battery_capacity_kwh": np.random.choice([0, 5, 10, 13], size=len(customer_ids), p=[0.85, 0.1, 0.03, 0.02]),
        "ev_plan_enrolled": np.random.choice([0, 1], len(customer_ids), p=[0.92, 0.08]),
        "midday_saver_enrolled": np.random.choice([0, 1], len(customer_ids), p=[0.7, 0.3]),
        "smart_meter_status": np.random.choice(["yes", "no"], len(customer_ids), p=[0.3, 0.7]),
    })

    df.to_csv(f"{OUTPUT_DIR}/technology_adoption.csv", index=False)
    print("Saved technology_adoption.csv")
    return df



# ============================================================
# E. WESTERN POWER NETWORK DATA (substation / feeder)
# ============================================================

def simulate_network_load_profiles():
    substations = 25
    records = []

    for s in range(substations):
        for day in range(365):
            for i in range(48):  # 30-min intervals
                load = np.random.normal(10_000, 2_000)  # kW load at substation
                records.append([s, day, i, max(0, load)])

    df = pd.DataFrame(records,
                      columns=["substation_id", "day_of_year", "interval_30min", "load_kw"])

    df.to_csv(f"{OUTPUT_DIR}/network_load_profiles.csv", index=False)
    print("Saved network_load_profiles.csv")
    return df



# ============================================================
# RUN EVERYTHING
# ============================================================

if __name__ == "__main__":
    billing_df = simulate_billing_system_data()

    smart_meter_ids = np.random.choice(billing_df["customer_id"], N_SMART_METERS, replace=False)
    interval_df = simulate_interval_data(smart_meter_ids)

    regulatory_df = simulate_regulatory_reporting()
    tech_df = simulate_technology_adoption(billing_df["customer_id"])
    network_df = simulate_network_load_profiles()

    print("All synthetic data generated in:", OUTPUT_DIR)


Saved billing_system_data.csv
Saved advanced_interval_data.csv
Saved regulatory_reporting.csv
Saved technology_adoption.csv
Saved network_load_profiles.csv
All synthetic data generated in: synthetic_synergy_data


In [11]:
billing_df.head()

,customer_id,postcode,suburb_code,tariff,customer_type,monthly_consumption_kwh,bill_amount,payment_history_score,concession_status,connection_date,customer_tenure_years
0,1,6102,100,A1,large_business,214.540771,439.460982,0.213238,1,2014-09-10,10.317808
1,2,6435,384,L1,small_business,299.118885,559.032124,0.077685,0,2094-10-04,-69.802740
2,3,6860,455,M1,residential,665.443131,439.120229,0.697628,0,2053-07-30,-28.594521
3,4,6270,144,L3,medium_business,187.480672,422.690873,0.987886,0,2075-08-01,-50.613699
4,5,6106,192,K1,medium_business,44.408474,178.543948,0.546124,0,2046-07-19,-21.558904


In [13]:
# using the data from the actual synergy system to fix the simulaition parameters

In [ ]:
"""
generate_synthetic_synergy.py

Generates realistic synthetic energy datasets (billing, interval, technology, network, regulatory)
with temperature-dependent demand curves and associated features.

Outputs (in OUTPUT_DIR):
 - billing_system.parquet          (per-customer monthly / summary fields)
 - billing_system_sample.csv       (small CSV sample for quick inspection)
 - interval_data.parquet           (subset customers: 30-min intervals, multi-day or multi-year)
 - technology_adoption.parquet
 - network_load_profiles.parquet
 - regulatory_reporting.csv

Configuration:
 - N_CUSTOMERS: change to 1_170_000 for full Synergy-scale simulated customer base.
 - INTERVAL_MINUTES: 30 (use 15 for higher resolution)
 - INTERVAL_DAYS: number of days to simulate for interval dataset (e.g. 365)
 - SMART_METER_SHARE: fraction of customers with interval data

Notes:
 - For N_CUSTOMERS >> 200k consider running with Dask or PySpark and/or chunked Parquet writes.
 - The model is deliberately modular: tweak coefficients to match desired realism.
"""

import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import math
import uuid
import multiprocessing as mp

# ----------------------------
# Configuration
# ----------------------------
OUTPUT_DIR = "synthetic_synergy_data_temp_model"
os.makedirs(OUTPUT_DIR, exist_ok=True)

N_CUSTOMERS = 100_000        # set to 1_170_000 for full scale (watch memory / use chunking)
SMART_METER_SHARE = 0.10     # fraction of customers with interval data (e.g. 10%)
INTERVAL_MINUTES = 30        # 30-minute intervals
INTERVAL_DAYS = 365          # days of interval data to simulate
PARQUET_ENGINE = "pyarrow"   # ensure pyarrow installed for parquet writes

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Geographic base: sample postcodes in Perth area (6000-6999)
POSTCODE_MIN = 6000
POSTCODE_MAX = 6999
N_POSTCODE_CLUSTERS = 50     # cluster postcodes to share weather patterns

# Date range for interval simulation
INTERVAL_START = datetime(2024, 1, 1)
INTERVAL_END = INTERVAL_START + timedelta(days=INTERVAL_DAYS)

# Temperature model parameters (Perth-like seasonal amplitude)
TEMP_MEAN = 21.0             # annual mean temperature (°C)
TEMP_AMP = 8.0               # amplitude of seasonal variation
TEMP_DAY_AMP = 5.0           # daily cycle amplitude
TEMP_NOISE_STD = 1.5         # hourly noise std dev

# Comfort temperature: demand increases above/below this for cooling/heating
COMFORT_TEMP = 20.0

# Tariffs pool (example)
TARIFFS = ["A1", "L1", "L3", "R1", "R3", "M1", "K1"]

# EV and PV probabilities
BASE_PV_PROB = 0.18        # fraction of households with PV
BASE_BATTERY_PROB = 0.05
BASE_EV_PROB = 0.08

# Interval scaling (kW per interval baseline)
BASE_DAILY_ENERGY_KWH_MEAN = 18.0   # typical household monthly ~540 kWh => daily ~18 kWh
BASE_DAILY_ENERGY_STD = 6.0

# ----------------------------
# Utilities
# ----------------------------
def make_customer_table(n_customers):
    """Create synthetic customer-level table with demographics and static features."""
    cust_ids = np.arange(1, n_customers + 1, dtype=np.int64)
    postcodes = np.random.randint(POSTCODE_MIN, POSTCODE_MAX + 1, size=n_customers)
    tariff = np.random.choice(TARIFFS, size=n_customers, p=[0.25,0.1,0.05,0.25,0.1,0.2,0.05])
    cust_type = np.random.choice(["residential","small_business","medium_business","large_business","home_business"],
                                 size=n_customers, p=[0.80,0.10,0.06,0.02,0.02])
    # customer tenure (years)
    connection_year = np.random.randint(1990, 2024, size=n_customers)
    connection_date = pd.to_datetime(connection_year.astype(str) + "-01-01") + \
                      pd.to_timedelta(np.random.randint(0,365,size=n_customers), unit='D')

    # baseline daily consumption (kWh/day) per customer
    daily_kwh = np.clip(np.random.normal(BASE_DAILY_ENERGY_KWH_MEAN, BASE_DAILY_ENERGY_STD, size=n_customers), 2.0, 200.0)

    # Temperature sensitivity coefficients: cooling (positive when temp>comfort), heating (positive when temp<comf)
    # Make distribution: most have small sensitivity; some tech/elderly households may have larger.
    cooling_coeff = np.random.normal(0.03, 0.015, size=n_customers)  # kWh per degC above comfort per hour
    heating_coeff = np.random.normal(0.02, 0.01, size=n_customers)   # kWh per degC below comfort per hour
    # ensure non-negative
    cooling_coeff = np.clip(cooling_coeff, 0.0, None)
    heating_coeff = np.clip(heating_coeff, 0.0, None)

    # Technology adoption flags (probabilistic by postcode cluster or tariff)
    pv_flag = np.random.binomial(1, BASE_PV_PROB, size=n_customers)
    battery_flag = (np.random.binomial(1, BASE_BATTERY_PROB, size=n_customers) & (pv_flag==1)).astype(int)
    ev_flag = np.random.binomial(1, BASE_EV_PROB, size=n_customers)

    # PV and battery sizes (if installed)
    pv_kw = pv_flag * np.random.choice([1.5,2.5,3.0,5.0,6.0,8.0], size=n_customers, p=[0.1,0.2,0.25,0.25,0.15,0.05])
    battery_kwh = battery_flag * np.random.choice([3.5,5.0,10.0,13.5], size=n_customers, p=[0.4,0.35,0.15,0.1])

    # EV characteristics
    ev_kwh_per_session = ev_flag * np.random.choice([7,10,12,15,20], size=n_customers, p=[0.2,0.25,0.3,0.15,0.1])
    ev_smart = ev_flag * np.random.binomial(1, 0.6, size=n_customers)  # share of EV owners with smart chargers

    # Smart thermostats adoption
    smart_thermostat = np.random.binomial(1, 0.18, size=n_customers)

    df = pd.DataFrame({
        "customer_id": cust_ids,
        "postcode": postcodes,
        "tariff": tariff,
        "customer_type": cust_type,
        "connection_date": connection_date,
        "daily_kwh_mean": daily_kwh,
        "cooling_coeff": cooling_coeff,
        "heating_coeff": heating_coeff,
        "pv_kw": pv_kw,
        "battery_kwh": battery_kwh,
        "ev_flag": ev_flag,
        "ev_kwh_per_session": ev_kwh_per_session,
        "ev_smart": ev_smart,
        "smart_thermostat": smart_thermostat,
    })
    return df

# ----------------------------
# Weather and temperature generation
# ----------------------------
def generate_temperature_series(start, end, freq_minutes=60, postcode_clusters=50):
    """
    Generate hourly (or freq_minutes) temperature time series per postcode cluster.
    Returns: dict(cluster_id -> pandas.Series(datetime index))
    Model: annual sinusoid + daily sinusoid + gaussian noise.
    """
    rng = pd.date_range(start=start, end=end - timedelta(minutes=1), freq=f"{freq_minutes}T", tz=None)
    n = len(rng)
    # Day of year in radians for annual cycle
    day_of_year = rng.dayofyear.values
    hour_of_day = rng.hour.values + rng.minute.values / 60.0

    temp_series_by_cluster = {}
    for c in range(postcode_clusters):
        # small variation by cluster (coastal vs inland)
        cluster_mean = TEMP_MEAN + np.random.normal(0, 1.0)
        cluster_amp = TEMP_AMP + np.random.normal(0, 1.5)
        # phase shift to represent locality
        phase_shift = np.random.uniform(0, 2*math.pi)
        annual = cluster_amp * np.sin(2 * math.pi * (day_of_year / 365.0) + phase_shift)
        daily = TEMP_DAY_AMP * np.sin(2 * math.pi * (hour_of_day / 24.0) + np.random.uniform(0, 2*math.pi))
        noise = np.random.normal(0, TEMP_NOISE_STD, size=n)
        temps = cluster_mean + annual + daily + noise
        temp_series_by_cluster[c] = pd.Series(temps, index=rng)
    return temp_series_by_cluster

# ----------------------------
# Solar generation model (simple) - per hour or interval
# ----------------------------
def simple_pv_generation(irradiance, pv_kw, system_capacity_factor=0.75):
    """
    irradiance: array representing normalized irradiance [0..1] for each time index (clear-sky proxy)
    pv_kw: PV system size in kW
    returns: kWh produced per interval (assuming irradiance is fraction of peak and interval in hours)
    system_capacity_factor: accounts for panel orientation, inverter efficiency
    """
    # irradiance is fraction of peak. For interval length h, energy = pv_kw * irradiance * h * eff
    return pv_kw * irradiance * system_capacity_factor

def make_normalized_irradiance_index(t_index):
    """
    Create normalized irradiance [0..1] for each timestamp (solar elevation proxy).
    We'll use hour of day and day of year to shape the irradiance curve (no atmospheric modeling).
    """
    doy = t_index.dayofyear.values
    hour = t_index.hour.values + t_index.minute.values / 60.0
    # approximate sunrise/sunset: depends on season -> use sun angle proxy
    # simple model: peak irradiance at noon, width depends on day length (approx)
    # daylength factor: longer in summer -> more hours with irradiance
    daylength = 10 + 4 * np.sin(2 * np.pi * (doy / 365.0))  # approx 6-14 hours mapped to 6..14
    # compute 'distance' from solar noon and apply gaussian shaped curve
    solar_noon = 12.0
    dt = (hour - solar_noon)
    sigma = daylength / 6.0  # controls width
    base = np.exp(-0.5 * (dt/sigma)**2)
    # zero out during night (when |hour - 12| > daylength/2)
    base[np.abs(dt) > (daylength/2.0)] = 0.0
    # scale to [0,1]
    maxv = base.max() if base.max() > 0 else 1.0
    norm = base / maxv
    return norm

# ----------------------------
# Interval demand generation
# ----------------------------
def generate_interval_profiles(customer_df, temp_series_by_cluster, interval_minutes=30, start=INTERVAL_START, days=INTERVAL_DAYS):
    """
    Vectorized-ish generation of interval demand for a subset of customers.
    Returns a DataFrame with columns: customer_id, timestamp, consumption_kwh, solar_export_kwh, battery_dispatch_kwh, ev_charging_kwh
    Note: for memory reasons, we only generate for customers passed in customer_df (a subset for smart meters).
    """
    # Create timestamps index
    rng = pd.date_range(start=start, periods=int(24*60/interval_minutes*days), freq=f"{interval_minutes}T")
    n_times = len(rng)
    # Map each customer's postcode -> cluster id
    # create postcode clusters mapping
    unique_postcodes = customer_df["postcode"].unique()
    postcode_to_cluster = {}
    # naive mapping: bucket postcodes into N_POSTCODE_CLUSTERS
    cluster_edges = np.linspace(POSTCODE_MIN, POSTCODE_MAX+1, N_POSTCODE_CLUSTERS+1)
    for pc in unique_postcodes:
        idx = np.searchsorted(cluster_edges, pc, side='right') - 1
        idx = max(0, min(N_POSTCODE_CLUSTERS-1, idx))
        postcode_to_cluster[pc] = idx

    # prepare irradiance for the timestamp index using normalized irradiance
    irradiance = make_normalized_irradiance_index(rng)

    # Precompute temperature series per cluster for the timestamps
    temp_matrix = np.zeros((N_POSTCODE_CLUSTERS, n_times), dtype=np.float32)
    for c in range(N_POSTCODE_CLUSTERS):
        s = temp_series_by_cluster[c]
        # Align s to rng index (s might have same freq)
        temp_matrix[c, :] = s.reindex(rng, method="nearest").values

    # For each customer, compute baseline interval consumption distribution
    # We'll create a daily shape profile (24h) normalized to 1, then scale by customer's daily kWh mean
    # daily profile: low at night, morning peak, daytime dip, evening peak
    hour = rng.hour + rng.minute / 60.0
    # create normalized load shape for a single day (periodic every 24h)
    # baseline shape (arbitrary but plausible)
    base_shape = 0.6 + 0.4*np.exp(-0.5*((hour-7)/2.0)**2) + 0.8*np.exp(-0.5*((hour-19)/3.0)**2) \
                 + 0.2*np.exp(-0.5*((hour-13)/3.0)**2)
    # normalize per day: we want sum over 24h to equal 1 for per-day scaling (account for interval length)
    # compute per-interval hours
    hrs_per_interval = interval_minutes / 60.0
    # group into days to normalize per day: we can compute normalization factor per day index
    # simpler: compute a typical day sum over 24h cycles
    # find positions for one representative day (first 24h)
    rep_day_mask = (rng.date == rng.date[0])
    # But easier: compute normalization per 24h cycle by reshaping
    # create array grouped by day
    days_count = int(n_times / (24*60/interval_minutes))
    shape_by_day = base_shape.reshape(days_count, -1)  # (days_count, intervals_per_day)
    # compute normalization (sum across intervals * hrs_per_interval)
    norm_per_day = shape_by_day.sum(axis=1) * hrs_per_interval
    # we'll divide each day's entries by that day's normalization
    normalized_shape = np.repeat((shape_by_day / norm_per_day[:, None]).reshape(-1), 1)

    # Prepare output container as list of DataFrames per chunk
    rows = []
    # To avoid huge memory use, process customers in chunks
    CHUNK = 2000  # tune based on memory
    cust_ids = customer_df["customer_id"].values
    n_cust = len(cust_ids)
    for i0 in range(0, n_cust, CHUNK):
        i1 = min(n_cust, i0 + CHUNK)
        chunk = customer_df.iloc[i0:i1]
        # For each customer in chunk, compute consumption over time vectorized-ish
        # Compute daily kwh to interval kwh: daily_kwh_mean * normalized_shape * randomness * temperature effect
        # Build matrix: (n_customers_in_chunk, n_times)
        ck = len(chunk)
        # repeat arrays
        daily_kwh = chunk["daily_kwh_mean"].values.reshape(-1,1)  # (ck,1)
        # baseline profile repeated
        profile = normalized_shape.reshape(1,n_times)  # (1, n_times)
        # randomness per customer and per time
        noise = np.random.normal(1.0, 0.05, size=(ck, n_times))

        # temperature effect: for each customer, map to cluster temps and compute deviation from comfort
        cluster_ids = np.array([postcode_to_cluster[pc] for pc in chunk["postcode"].values], dtype=np.int32)
        temp_for_customers = temp_matrix[cluster_ids, :]  # (ck, n_times)
        # cooling effect: when temp > COMFORT_TEMP
        temp_dev_cooling = np.clip(temp_for_customers - COMFORT_TEMP, 0, None)
        temp_dev_heating = np.clip(COMFORT_TEMP - temp_for_customers, 0, None)
        # per-hour effect convert coeffs (kWh per degC per hour) to per-interval
        cooling_coeff = chunk["cooling_coeff"].values.reshape(-1,1)
        heating_coeff = chunk["heating_coeff"].values.reshape(-1,1)
        temp_effect = (cooling_coeff * temp_dev_cooling + heating_coeff * temp_dev_heating) * (interval_minutes/60.0)

        # compute base consumption
        consumption_kwh = daily_kwh * profile * noise + temp_effect

        # solar export per customer: compute irradiance * pv_kw * eff * interval_hours
        pv_kw = chunk["pv_kw"].values.reshape(-1,1)
        irradiance_arr = irradiance.reshape(1, -1)  # (1,n_times)
        pv_eff = 0.75
        solar_kwh = pv_kw * irradiance_arr * pv_eff * (interval_minutes/60.0)

        # battery simple dispatch: households with battery will absorb midday solar up to battery capacity then discharge in evening
        battery_kwh = np.zeros_like(consumption_kwh)
        has_batt = chunk["battery_kwh"].values
        # simple heuristic per-customer battery usage across day: charge during intervals where solar>0, discharge during evening peak hours (17-21)
        # Implement per-day dispatch in simplistic manner to avoid complex optimization
        if np.any(has_batt > 0):
            batt_caps = chunk["battery_kwh"].values
            for idx, cap in enumerate(batt_caps):
                if cap <= 0:
                    continue
                # per-day schedule: charge from solar up to cap, then discharge uniformly in 17-21 hours
                # compute per-day arrays
                # reshape to days x intervals_per_day
                ints_per_day = int(24*60/INTERVAL_MINUTES)
                cons_daily = consumption_kwh[idx].reshape(days_count, ints_per_day)
                solar_daily = solar_kwh[idx].reshape(days_count, ints_per_day)
                batt_daily = np.zeros_like(cons_daily)
                for d in range(days_count):
                    available_charge = solar_daily[d].sum()  # naive
                    charge_amt = min(cap, available_charge * 0.9)  # fraction can be stored
                    # charge during daytime intervals proportional to solar
                    if solar_daily[d].sum() > 0:
                        charge_profile = solar_daily[d] / solar_daily[d].sum() * charge_amt
                    else:
                        charge_profile = np.zeros_like(solar_daily[d])
                    # discharge in evening intervals (17:00-21:00)
                    evening_mask = ((np.arange(ints_per_day) * (INTERVAL_MINUTES/60.0)) >= 17) & ((np.arange(ints_per_day)*(INTERVAL_MINUTES/60.0)) < 21)
                    if evening_mask.sum() > 0:
                        discharge_profile = np.zeros(ints_per_day)
                        discharge_profile[evening_mask] = charge_amt / evening_mask.sum()
                    else:
                        discharge_profile = np.zeros(ints_per_day)
                    batt_daily[d] = - discharge_profile + charge_profile  # positive = charge, negative = discharge (net to grid)
                battery_kwh[idx] = batt_daily.reshape(-1)
            # battery_kwh now contains positive for charge (consumes from grid) and negative for discharge (exports)
        # EV charging model: for ev owners, add charging demand typically at evening/night unless smart charging (shiftable)
        ev_charging_kwh = np.zeros_like(consumption_kwh)
        ev_flag_arr = chunk["ev_flag"].values
        ev_session = chunk["ev_kwh_per_session"].values
        ev_smart = chunk["ev_smart"].values
        # simplistic model: each EV does 0-1 sessions per day (prob 0.5), arrives at 18:00 and charges until filled unless smart scheduler shifts to off-peak
        ints_per_day = int(24*60/INTERVAL_MINUTES)
        for idx in range(ck):
            if ev_flag_arr[idx] == 0:
                continue
            for d in range(days_count):
                didx_start = d*ints_per_day + int((18.0 / 24.0) * ints_per_day)  # 18:00 index approximation
                energy_needed = ev_session[idx]
                if ev_smart[idx] == 1:
                    # smart behavior: shift to off-peak (assume off-peak midnight-6) or split
                    # we'll prefer midnight start if tariff encourages
                    start_idx = d*ints_per_day + int((1.0 / 24.0) * ints_per_day)  # 01:00
                else:
                    start_idx = didx_start
                # simple charging rate 7 kW -> per interval kWh
                per_interval_kwh = 7.0 * (INTERVAL_MINUTES/60.0)
                intervals_needed = int(max(1, math.ceil(energy_needed / per_interval_kwh)))
                for t in range(intervals_needed):
                    t_idx = start_idx + t
                    if t_idx < (d+1)*ints_per_day:
                        ev_charging_kwh[idx, t_idx] += per_interval_kwh
                    else:
                        break

        # Now compute net grid consumption: base consumption + battery_charge (positive) - battery_discharge (negative) + ev_charging - solar_export (solar exported if production exceeds consumption+charge)
        net_before_export = consumption_kwh + np.maximum(battery_kwh, 0) + ev_charging_kwh
        # solar_export: if solar production > net_before_export, export = surplus, else zero
        solar_export = np.maximum(solar_kwh - net_before_export, 0.0)
        # final grid consumption = net_before_export - min(solar_kwh, net_before_export)
        grid_consumption = net_before_export - np.minimum(solar_kwh, net_before_export)

        # Prepare DataFrame rows for chunk
        # We will store: customer_id, timestamp, grid_kwh, solar_kwh, solar_export_kwh, battery_kwh, ev_charging_kwh
        # To keep file size manageable, write per-customer compressed or per-chunk DataFrames and append to Parquet
        for j, cust in enumerate(chunk["customer_id"].values):
            row_df = pd.DataFrame({
                "customer_id": np.repeat(int(cust), n_times),
                "timestamp": rng,
                "grid_consumption_kwh": grid_consumption[j],
                "solar_kwh": solar_kwh[j],
                "solar_export_kwh": solar_export[j],
                "battery_dispatch_kwh": battery_kwh[j],   # positive charge, negative discharge
                "ev_charging_kwh": ev_charging_kwh[j],
            })
            # append to Parquet file (row-group/chunk)
            out_file = os.path.join(OUTPUT_DIR, "interval_data.parquet")
            # Use append mode by writing in first chunk then append
            if (i0 == 0 and j == 0 and not os.path.exists(out_file)):
                row_df.to_parquet(out_file, engine=PARQUET_ENGINE, index=False)
            else:
                # append by reading/writing with pyarrow backend would be better; pandas to_parquet does not support append - do chunked single-file approach:
                # Instead, write each chunk as separate file and later concatenate (recommended for large-scale)
                chunk_file = os.path.join(OUTPUT_DIR, f"interval_chunk_{i0}_{j}.parquet")
                row_df.to_parquet(chunk_file, engine=PARQUET_ENGINE, index=False)
        # end chunk loop
        print(f"Processed customers {i0} .. {i1-1} for interval profiles")

    # After generating many small parquet chunks, optionally concatenate them into one Parquet dataset folder.
    # For now, we leave many chunk files in OUTPUT_DIR; user can merge using pyarrow.dataset or fastparquet if desired.
    return True

# ----------------------------
# Network load profiles (aggregated)
# ----------------------------
def simulate_network_load_profiles(postcode_customer_df, interval_minutes=30, start=INTERVAL_START, days=INTERVAL_DAYS):
    """
    Aggregate simulated network load at zone substation level by summing a sampled fraction of customers.
    We'll create synthetic zone substations and produce 30-min aggregated load time series.
    """
    rng = pd.date_range(start=start, periods=int(24*60/interval_minutes*days), freq=f"{interval_minutes}T")
    ints_per_day = int(24*60/interval_minutes)
    days_count = int(len(rng) / ints_per_day)
    substations = 50
    # choose random assignment of postcodes to substations
    unique_postcodes = np.unique(postcode_customer_df["postcode"].values)
    pc_to_sub = {pc: int(hash(str(pc)) % substations) for pc in unique_postcodes}

    # baseline load per substation per interval (kW)
    records = []
    for s in range(substations):
        # create base profile using hour profile times a substation scale factor
        hour = rng.hour + rng.minute/60.0
        base_profile = 1000 * (1.0 + 0.5*np.exp(-0.5*((hour-7)/2.0)**2) + 0.8*np.exp(-0.5*((hour-19)/3.0)**2))
        scale = np.random.uniform(0.5, 2.0)  # vary by substation customer density
        noise = np.random.normal(0, 100, size=len(rng))
        load_kw = np.maximum(0, base_profile * scale + noise)
        df = pd.DataFrame({
            "substation_id": np.repeat(s, len(rng)),
            "timestamp": rng,
            "load_kw": load_kw
        })
        # write per-substation parquet to avoid memory pressure
        df.to_parquet(os.path.join(OUTPUT_DIR, f"network_substation_{s}.parquet"), engine=PARQUET_ENGINE, index=False)
        records.append({"substation_id": s, "avg_load_kw": load_kw.mean()})
    # write summary
    pd.DataFrame(records).to_csv(os.path.join(OUTPUT_DIR, "network_substation_summary.csv"), index=False)
    return True

# ----------------------------
# Regulatory reporting simulation
# ----------------------------
def simulate_regulatory_reporting(n_customers):
    # simplified ERA-like indicators for 3 years
    years = [2022, 2023, 2024]
    rows = []
    for y in years:
        rows.append({
            "year": y,
            "total_customers": n_customers,
            "residential": int(n_customers * 0.80),
            "business": int(n_customers * 0.20),
            "complaints": int(np.random.randint(4000, 20000)),
            "affordability_index": round(np.random.uniform(0.2, 0.8), 3),
            "disconnections": int(np.random.randint(1000, 8000)),
            "payment_difficulty": int(np.random.randint(4000, 20000)),
            "life_support_customers": int(np.random.randint(200, 2000)),
        })
    df = pd.DataFrame(rows)
    df.to_csv(os.path.join(OUTPUT_DIR, "regulatory_reporting.csv"), index=False)
    return df

# ----------------------------
# Billing (monthly summary) generation
# ----------------------------
def generate_billing_summary(customer_df, months=12):
    """
    Generate monthly billing system summary for each customer by sampling consumption from daily means and tariffs.
    Writes billing_system.parquet and a small CSV sample.
    """
    # simulate month-by-month totals for 'months' months
    months_idx = pd.date_range(end=datetime(2024,12,31), periods=months, freq='M')
    rows = []
    for i, row in customer_df.iterrows():
        cust_id = int(row["customer_id"])
        base_daily = float(row["daily_kwh_mean"])
        tariff = row["tariff"]
        concession = 0 if np.random.rand() > 0.12 else 1
        for m in months_idx:
            # seasonal multiplier by month (simple)
            month_season_factor = 1.0 + 0.15 * np.sin(2*math.pi*(m.month/12.0))
            # random monthly variation
            monthly_kwh = base_daily * 30 * month_season_factor * np.random.normal(1.0, 0.08)
            bill_amount = monthly_kwh * (0.30 + np.random.normal(0.0, 0.05))  # approximate $/kWh
            rows.append({
                "customer_id": cust_id,
                "invoice_month": m.strftime("%Y-%m"),
                "monthly_kwh": monthly_kwh,
                "bill_amount": round(bill_amount, 2),
                "tariff": tariff,
                "concession": concession,
                "postcode": row["postcode"]
            })
    billing_df = pd.DataFrame(rows)
    # write parquet (partition by invoice_month if desired)
    billing_df.to_parquet(os.path.join(OUTPUT_DIR, "billing_system.parquet"), engine=PARQUET_ENGINE, index=False)
    billing_df.sample(min(5000, len(billing_df))).to_csv(os.path.join(OUTPUT_DIR, "billing_system_sample.csv"), index=False)
    return billing_df

# ----------------------------
# Main script orchestration
# ----------------------------
def main():
    print("Generating customer table...")
    cust_df = make_customer_table(N_CUSTOMERS)
    cust_df.to_parquet(os.path.join(OUTPUT_DIR, "customers.parquet"), engine=PARQUET_ENGINE, index=False)
    print(f"Saved customers.parquet ({len(cust_df)} rows)")

    # generate postcode clusters temperature series
    print("Generating synthetic temperature series for postcode clusters...")
    temp_series_by_cluster = generate_temperature_series(start=INTERVAL_START, end=INTERVAL_END, freq_minutes=INTERVAL_MINUTES, postcode_clusters=N_POSTCODE_CLUSTERS)
    # Optionally persist temp series for reproducibility
    # For speed, store cluster means as csv (not the full timeseries)
    cluster_stats = []
    for cid, s in temp_series_by_cluster.items():
        cluster_stats.append({"cluster_id": cid, "mean_temp": float(s.mean()), "std_temp": float(s.std())})
    pd.DataFrame(cluster_stats).to_csv(os.path.join(OUTPUT_DIR, "temp_cluster_stats.csv"), index=False)

    # Technology adoption table
    print("Saving technology adoption...")
    tech_cols = ["customer_id","pv_kw","battery_kwh","ev_flag","ev_kwh_per_session","ev_smart","smart_thermostat"]
    tech_df = cust_df[["customer_id","pv_kw","battery_kwh","ev_flag","ev_kwh_per_session","ev_smart","smart_thermostat"]].copy()
    tech_df.to_parquet(os.path.join(OUTPUT_DIR, "technology_adoption.parquet"), engine=PARQUET_ENGINE, index=False)
    print("Saved technology_adoption.parquet")

    # Generate billing monthly summaries
    print("Generating billing monthly summaries (this may take a while depending on N_CUSTOMERS)...")
    billing_df = generate_billing_summary(cust_df, months=12)
    print("Saved billing_system.parquet and sample CSV")

    # Interval data for smart meter customers (subset)
    n_smart = int(N_CUSTOMERS * SMART_METER_SHARE)
    smart_customers = cust_df.sample(n=n_smart, random_state=RANDOM_SEED).reset_index(drop=True)
    smart_customers.to_parquet(os.path.join(OUTPUT_DIR, "smart_customers.parquet"), engine=PARQUET_ENGINE, index=False)
    print(f"Selected {n_smart} customers for interval simulation and saved smart_customers.parquet")

    print("Generating network load profiles (per-substation parquet files)...")
    simulate_network_load_profiles(cust_df)

    print("Generating regulatory reporting...")
    simulate_regulatory_reporting(N_CUSTOMERS)

    print("Starting interval profile generation (this can be slow; check OUTPUT_DIR for chunked parquet files)...")
    # For large datasets consider running generate_interval_profiles in parallel or with Dask
    generate_interval_profiles(smart_customers, temp_series_by_cluster, interval_minutes=INTERVAL_MINUTES, start=INTERVAL_START, days=INTERVAL_DAYS)

    print("All datasets generated in:", OUTPUT_DIR)
    print("Note: interval generation writes many chunk files. Use pyarrow.dataset or pandas.concat (careful with memory) to assemble a single dataset if needed.")

if __name__ == "__main__":
    main()
